In [11]:
import os
from google.cloud import bigquery
from google.cloud import aiplatform
import numpy as np
import pandas as pd
from datetime import datetime 

In [12]:
PROJECT = "pacific-torus-347809"
REGION = "asia-southeast1-a"

In [13]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../../pacific-torus.json"

In [14]:
%load_ext google.cloud.bigquery

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


In [15]:
bq = bigquery.Client(project=PROJECT)

# create dataset example
# dataset = bigquery.Dataset(bq.dataset("dataset-name"))
# try: 
#     bq.create_dataset(dataset) 
#     print("Dataset created")
# except: 
#     print("Dataset already exists")

In [8]:
%%bigquery 

SELECT DISTINCT
  id, 
  limit_balance, 
  sex,
  education_level, 
  marital_status,
  age,
  pay_0,
  pay_2, 
  pay_3,
  pay_4,
  pay_5, 
  pay_6, 
  bill_amt_1, 
  bill_amt_2,
  bill_amt_3,
  bill_amt_4, 
  bill_amt_5, 
  bill_amt_6, 
  pay_amt_1,
  pay_amt_2,
  pay_amt_3,
  pay_amt_4,
  pay_amt_5,
  pay_amt_6, 
  default_payment_next_month,
FROM `bigquery-public-data.ml_datasets.credit_card_default` 


Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.67s/rows]


,id,limit_balance,sex,education_level,marital_status,age,pay_0,pay_2,pay_3,pay_4,...,bill_amt_4,bill_amt_5,bill_amt_6,pay_amt_1,pay_amt_2,pay_amt_3,pay_amt_4,pay_amt_5,pay_amt_6,default_payment_next_month
0,242.0,50000.0,1,1,2,39.0,0.0,0.0,0.0,0.0,...,50803.0,30789.0,15874.0,1800.0,2000.0,3000.0,2000.0,2000.0,2000.0,0


In [5]:
sql_query = """
SELECT DISTINCT
  id, 
  limit_balance, 
  sex,
  education_level, 
  marital_status,
  age,
  pay_0,
  pay_2, 
  pay_3,
  pay_4,
  pay_5, 
  pay_6, 
  bill_amt_1, 
  bill_amt_2,
  bill_amt_3,
  bill_amt_4, 
  bill_amt_5, 
  bill_amt_6, 
  pay_amt_1,
  pay_amt_2,
  pay_amt_3,
  pay_amt_4,
  pay_amt_5,
  pay_amt_6, 
  default_payment_next_month,
FROM `bigquery-public-data.ml_datasets.credit_card_default` 
"""

# df = bq.query(sql_query).to_dataframe()

# df.head()

In [106]:
dest_project_id = ""

In [116]:
from pipelines.ingest.bq_extract_data import bq_extract_data

from pipelines.kfp.dependencies import PROJECT_ID

In [114]:
PROJECT_ID

'pacific-torus-347809'

In [115]:

dd, duri = bq_extract_data(
    source_project_id="bigquery-public-data",
    source_dataset_id="ml_datasets",
    dest_project_id='pacific-torus-347809',
    source_table_id="credit_card_default",
    dest_bucket="mle-dwh-torus",
    dest_file="raw/raw_data_backup.json",
    dataset_location="US",
)

26-05-2022 10:04:31 : root : INFO - Extract bigquery-public-data.ml_datasets.credit_card_default to gs://mle-dwh-torus/raw_data.json
26-05-2022 10:04:36 : root : INFO - Table extracted: bigquery-public-data.ml_datasets.credit_card_default


In [117]:
import logging
import logging.config
import os
import subprocess

from google.cloud import bigquery, storage
from google.cloud.exceptions import GoogleCloudError
from pipelines.kfp.dependencies import LOGGING_CONF
from pipelines.kfp.helpers import create_bucket, setup_credentials



bq_query_to_table(
    query=sql_query, 
    source_project_id="bigquery-public-data",
    source_dataset_id="ml_datasets",
    source_table_id="credit_card_default",
    dest_project_id="pacific-torus-347809",
    dest_bucket="mle-dwh-torus",
    dest_file="raw.json",
    dataset_location="US",
    query_job_config=None,
) 

In [9]:
setup_credentials()

In [30]:
logging.config.fileConfig(LOGGING_CONF)
logger = logging.getLogger("root")



In [118]:
dest_project_id

''

In [120]:
full_table_id = "bigquery-public-data.ml_datasets.credit_card_default"

In [151]:
pd.read_parquet("gs://mle-dwh-torus/raw_tst.parquet")

,id,limit_balance,sex,education_level,marital_status,age,pay_0,pay_2,pay_3,pay_4,...,bill_amt_5,bill_amt_6,pay_amt_1,pay_amt_2,pay_amt_3,pay_amt_4,pay_amt_5,pay_amt_6,default_payment_next_month,predicted_default_payment_next_month
0,242.0,50000.0,1,1,2,39.0,0.0,0.0,0.0,0.0,...,30789.0,15874.0,1800.0,2000.0,3000.0,2000.0,2000.0,2000.0,0,"[{'tables': {'score': 0.8505047559738159, 'val..."
1,1822.0,110000.0,2,1,2,29.0,0.0,0.0,0.0,0.0,...,26354.0,20221.0,2000.0,2010.0,3000.0,3000.0,3000.0,1000.0,0,"[{'tables': {'score': 0.9308085441589355, 'val..."
2,5046.0,270000.0,1,1,2,36.0,0.0,0.0,0.0,2.0,...,37165.0,22156.0,10076.0,4018.0,14.0,2051.0,2000.0,0.0,0,"[{'tables': {'score': 0.8745065331459045, 'val..."
3,7227.0,130000.0,1,1,1,45.0,0.0,0.0,0.0,0.0,...,63832.0,65099.0,2886.0,2908.0,2129.0,2354.0,2366.0,2291.0,0,"[{'tables': {'score': 0.9030028581619263, 'val..."
4,9183.0,50000.0,1,1,2,24.0,0.0,0.0,0.0,0.0,...,19855.0,20151.0,1367.0,1606.0,692.0,709.0,721.0,692.0,0,"[{'tables': {'score': 0.856909453868866, 'valu..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2960,6203.0,210000.0,2,2,1,39.0,-1.0,-1.0,-1.0,-1.0,...,0.0,12000.0,0.0,28428.0,0.0,0.0,12000.0,33664.0,0,"[{'tables': {'score': 0.8730423450469971, 'val..."
2961,6067.0,280000.0,2,1,1,41.0,2.0,-1.0,-1.0,-1.0,...,0.0,244.0,6859.0,486.0,0.0,0.0,244.0,0.0,0,"[{'tables': {'score': 0.5389663577079773, 'val..."
2962,24215.0,80000.0,1,1,2,28.0,1.0,-1.0,-1.0,-1.0,...,0.0,0.0,47.0,207.0,0.0,0.0,0.0,0.0,0,"[{'tables': {'score': 0.6380036473274231, 'val..."
2963,18267.0,200000.0,1,2,1,53.0,1.0,-1.0,-1.0,-2.0,...,0.0,6753.0,2920.0,0.0,0.0,0.0,6753.0,0.0,1,"[{'tables': {'score': 0.6638315916061401, 'val..."


In [41]:
from pipelines.ingest.bq_query_data import bq_query_to_table
from pipelines.kfp.dependencies import PROJECT_ID

In [44]:
sql_query = """
SELECT DISTINCT
  id, 
  limit_balance, 
  sex,
  education_level, 
  marital_status,
  age,
  pay_0,
  pay_2, 
  pay_3,
  pay_4,
  pay_5, 
  pay_6, 
  bill_amt_1, 
  bill_amt_2,
  bill_amt_3,
  bill_amt_4, 
  bill_amt_5, 
  bill_amt_6, 
  pay_amt_1,
  pay_amt_2,
  pay_amt_3,
  pay_amt_4,
  pay_amt_5,
  pay_amt_6, 
  default_payment_next_month,
FROM `bigquery-public-data.ml_datasets.credit_card_default` 
"""

bq_query_to_table(
    query=sql_query, 
    bq_client_project_id = None,
    destination_project_id = PROJECT_ID,
    dataset_id = "dwh_pacific_torus",
    table_id = "credit_card_default",
    dataset_location = "US",
    query_job_config = None,
) 

TypeError: bq_query_to_table() got an unexpected keyword argument 'bq_client_project_id'

In [24]:
from google.cloud import bigquery

In [38]:
bq_client = bigquery.Client(location='US')


In [39]:
job_config = bigquery.QueryJobConfig(destination=f"{PROJECT_ID}.dwh_pacific_torus.credit_card_default")

In [40]:
bq_client.query(sql_query, job_config=job_config)

In [9]:
import pandas as pd

In [10]:
df = pd.read_parquet("gs://mle-dwh-torus/raw/credit_cards.parquet")

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2965 entries, 0 to 2964
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          2965 non-null   float64
 1   limit_balance               2965 non-null   float64
 2   sex                         2965 non-null   object 
 3   education_level             2965 non-null   object 
 4   marital_status              2965 non-null   object 
 5   age                         2965 non-null   float64
 6   pay_0                       2965 non-null   float64
 7   pay_2                       2965 non-null   float64
 8   pay_3                       2965 non-null   float64
 9   pay_4                       2965 non-null   float64
 10  pay_5                       2965 non-null   object 
 11  pay_6                       2965 non-null   object 
 12  bill_amt_1                  2965 non-null   float64
 13  bill_amt_2                  2965 